In [1]:
%cd ../../..

/home/user/Projects/denk_baseline


In [2]:
import os
import glob
from pprint import pprint

import torch
import pandas as pd
from omegaconf import OmegaConf

from denk_baseline.datamodules import DataModule
# from denk_baseline.custom_models.classification.next_vit import NextViT
from denk_baseline.lightning_models import ClassificationBinaryModel
from run import preprocess_config, parse_loggers, get_obj_from_str

/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config = OmegaConf.load('./rsna_config.yaml')
config = preprocess_config(config)

In [4]:
config['datasets']['test'] = {}
config['datasets']['test']['target'] = 'projects.rsna_2022.dataset.RSNADataset'
config['datasets']['test']['params'] = {
    'images_dir': '/media/user/FastNVME/processed_images', 
    'csv_path': '/home/user/datasets/rsna_2022/folds/holdt.csv', 
    'stage': 'test',
    'img_h': 512*3, 
    'img_w': 320*3,
}

In [5]:
datamodule = DataModule(config)

/home/user/Projects/denk_baseline/projects/rsna_2022/dataset.py:14: DtypeWarning: Columns (2,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(csv_path).reset_index()[:-2]


In [6]:
out_df = pd.DataFrame(columns=['name', 'loss', 'pre', 'rec', 'f1', 'pf', 'pf_tres', 'auc', 'tres'])

exp_name = config['common']['exp_name']
project_name = config['common']['project_name']
save_dir = config['common']['save_dir']

exp_dir = f'./{save_dir}/{project_name}/{exp_name}'
ckpt_paths = glob.glob(exp_dir + '/*.ckpt')

test_models_folder = 'tested_models'

ckpt_paths = list(sorted(ckpt_paths))

for ckpt_path in ckpt_paths:
    if 'last' in os.path.basename(ckpt_path):
        continue
    
    print(ckpt_path)
    model = ClassificationBinaryModel(config)
    model.load_state_dict(torch.load(ckpt_path, map_location='cpu')['state_dict'])

    trainer = get_obj_from_str(config['trainer']['target'])(**config['trainer']['params'])
    result = trainer.test(model, datamodule=datamodule)

    state_dict = {}
    m_dict = model.state_dict()
    for name in m_dict:
        state_dict[name.replace('model.', '')] = m_dict[name]
    
    b_metric = os.path.basename(ckpt_path[:-4]).split('-')[0]
    loss = result[0]['total_loss_test']
    pfscore = result[0]['pfscore_test']
    auc = result[0]['roc_auc_torch_test']
    
    out_df = out_df.append({
        'name': b_metric,
        'loss': result[0]['total_loss_test'], 
        'pre': result[0]['pre_torch_test'], 
        'rec': result[0]['rec_torch_test'], 
        'f1': result[0]['f1_torch_test'], 
        'pf': result[0]['pfscore_test'], 
        'pf_tres': result[0]['pfscore_thres_test'], 
        'auc': result[0]['roc_auc_torch_test'], 
        'tres': result[0]['test_best_threshold'],
    }, ignore_index=True)
    
    os.makedirs(f'./{save_dir}/{test_models_folder}/{exp_name}', exist_ok=True)
    
    out_path = f'./{save_dir}/{test_models_folder}/{exp_name}/nextvit-0-{b_metric}-loss-{loss:3.3f}-auc-{auc:3.3f}-pfscore-{pfscore:3.3f}.ckpt'
    torch.save({
        'state_dict': state_dict,
    }, out_path)

./output/rsna-2022/eff3-1536-960-0-0-no-kornia/best_auc-epoch=021-pfscore_valid=0.269-roc_auc_torch_valid=0.809-total_loss_valid=0.133.ckpt


/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████| 689/689 [01:59<00:00,  5.78it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      bce_loss_test         0.16234838962554932
      f1_torch_test         0.16091954708099365
      pfscore_test          0.1724127233028412
   pfscore_thres_test       0.22047244012355804
     pre_torch_test         0.3684210479259491
     rec_torch_test         0.10294117778539658
   roc_auc_torch_test       0.7923235297203064
   test_best_threshold      0.10000000149011612
     total_loss_test        0.16234838962554932
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
./output/rsna-2022/eff3-1536-960-0-0-no-kornia/best_loss-epoch=006-pfscore_valid=0.

/tmp/ipykernel_1366452/4032140444.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append({
/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████| 689/689 [01:57<00:00,  5.86it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      bce_loss_test         0.10717234760522842
      f1_torch_test         0.08163265883922577
      pfscore_test          0.08932296931743622
   pfscore_thres_test       0.15999998152256012
     pre_torch_test         0.5454545617103577
     rec_torch_test         0.04411764815449715
   roc_auc_torch_test        0.739054799079895
   test_best_threshold      0.10000000149011612
     total_loss_test        0.10717234760522842
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/tmp/ipykernel_1366452/4032140444.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append({


./output/rsna-2022/eff3-1536-960-0-0-no-kornia/best_pfscore-epoch=008-pfscore_valid=0.275-roc_auc_torch_valid=0.774-total_loss_valid=0.151.ckpt


/home/user/Projects/denk_baseline/venv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|███████████████████| 689/689 [02:00<00:00,  5.74it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      bce_loss_test         0.18172331154346466
      f1_torch_test         0.16149069368839264
      pfscore_test          0.1385989934206009
   pfscore_thres_test       0.1589958220720291
     pre_torch_test         0.5199999809265137
     rec_torch_test         0.09558823704719543
   roc_auc_torch_test       0.7646257877349854
   test_best_threshold      0.10000000149011612
     total_loss_test        0.18172331154346466
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/tmp/ipykernel_1366452/4032140444.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append({


In [7]:
out_df

,name,loss,pre,rec,f1,pf,pf_tres,auc,tres
0,best_auc,0.162348,0.368421,0.102941,0.160920,0.172413,0.220472,0.792324,0.1
1,best_loss,0.107172,0.545455,0.044118,0.081633,0.089323,0.160000,0.739055,0.1
2,best_pfscore,0.181723,0.520000,0.095588,0.161491,0.138599,0.158996,0.764626,0.1
